In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('data.csv', index_col=False)
df = df[df['deviceuuid'] == 'aNyZC8jwH3']
df = df[['create_time','weight', 'skeletal_muscle_mass', 'body_fat_mass', 'body_fat']]
df['create_time'] = pd.to_datetime(df['create_time'])
df = df.sort_values(by='create_time')
# group by day
df['day'] = df['create_time'].dt.date
df = df.groupby('day').mean()
df = df.reset_index()
df = df.drop(columns=['create_time'])
df = df.round(2)
df.to_csv('proc_data.csv', index=False)
df['day'] = pd.to_datetime(df['day'])
df = df[df['day'] >= datetime(2024, 3, 1)]
# df = df[df['day'] <= datetime(2024, 12, 1)]
print('last date', df['day'].max())
df = df.rename(columns={'body_fat': 'body_fat_percentage', 'skeletal_muscle_mass': 'muscle_mass'})
# Calculate monthly means and standard deviations
df_monthly = df.groupby(pd.Grouper(key='day', freq='ME')).agg({
    'weight': ['mean', 'std'],
    'muscle_mass': ['mean', 'std'],
    'body_fat_mass': ['mean', 'std'],
    'body_fat_percentage': ['mean', 'std']
})

# Flatten column names and reset index
df_monthly.columns = ['_'.join(col).strip() for col in df_monthly.columns]
df_monthly = df_monthly.reset_index()
df_monthly = df_monthly.round(2)
df_monthly['label'] = df_monthly['day'].dt.strftime('%Y-%m')

# Calculate month-to-month variations
for metric in ['weight', 'muscle_mass', 'body_fat_mass', 'body_fat_percentage']:
    mean_col = f'{metric}_mean'
    df_monthly[f'{metric}_variation'] = df_monthly[mean_col].diff()
# replace nan with 0
df_monthly = df_monthly.fillna(0)
df_monthly['muscle_mass_percentage_mean'] = df_monthly['muscle_mass_mean'] / df_monthly['weight_mean'] * 100
df_monthly['body_fat_mass_percentage_mean'] = df_monthly['body_fat_mass_mean'] / df_monthly['weight_mean'] * 100
df_monthly = df_monthly.round(2)
df_monthly.tail()

In [3]:
def plot_monthly_variation(df_monthly:pd.DataFrame, metric:str, metric_label:str, positive: bool = True):
    # Create figure and set background color
    fig, ax = plt.subplots(figsize=(12, 12))
    fig.patch.set_facecolor('#13181c')
    ax.set_facecolor('#13181c')

    # Remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Style the remaining spines
    ax.spines['bottom'].set_color('gray')
    ax.spines['left'].set_color('gray')

    # Convert dates to numeric values for regression
    x = np.arange(len(df_monthly))
    y = df_monthly[f'{metric}_mean']
    y_err = df_monthly[f'{metric}_std']

    # Calculate trend line
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    trend_line = p(x)
    # calculate the slope of the trend line
    slope = z[0]

	# Plot error bars
    ax.errorbar(df_monthly['label'], y, yerr=y_err,
                capsize=5, color='#0767e6', alpha=0.3)
    # Plot trend line
    ax.plot(df_monthly['label'], trend_line, '--', color='#e67607',
            label=f'Trend {slope:.2f} Kg/month', alpha=0.3)
	# plot line
    ax.plot(df_monthly['label'], df_monthly[f'{metric}_mean'],
			label=f'{metric_label}', marker='o',
			markerfacecolor='#13181c', markeredgecolor='#0767e6',
			color='#0767e6')


    # Add variation labels with colored backgrounds
    for i, (date, value, variation) in enumerate(zip(df_monthly['label'],
                                            df_monthly[f'{metric}_mean'],
                                            df_monthly[f'{metric}_variation'])):
            if pd.notna(variation) and i > 0:
                    if positive:
                        bg_color = '#07b015' if variation > 0 else '#8f0715'
                        xy=(date, value+0.01) if variation > 0 else (date, value-0.12)
                    else:
                        bg_color = '#8f0715' if variation > 0 else '#07b015'
                        xy=(date, value+0.1) if variation > 0 else (date, value-0.1)
                    text_color = 'white'

                    ax.annotate(f'{variation:+.2f}',
                                    xy=xy,
                                    xytext=(0, 10),
                                    textcoords='offset points',
                                    ha='center',
                                    color=text_color,
                                    bbox=dict(boxstyle='round,pad=0.5',
                                            fc=bg_color,
                                            ec=None,
                                            alpha=0.85))

    # Style the grid and labels
    ax.grid(True, linestyle='--', alpha=0.2, color='gray')
    ax.set_title(f'Average {metric_label} Per Month', pad=20, color='white')
    ax.set_ylabel(f'{metric_label} (kg)', color='white')
    ax.tick_params(colors='white')
    ax.legend(facecolor='#13181c', edgecolor='gray', labelcolor='white')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{metric_label}_monthly.png', dpi=300)
    plt.show()


In [ ]:
plot_monthly_variation(df_monthly, 'muscle_mass', 'Muscle Mass')

In [ ]:
plot_monthly_variation(df_monthly, 'body_fat_percentage', 'Body Fat Percentage', positive=False)

In [ ]:
df_monthly['muscle_mass_mean'].max() - df_monthly['muscle_mass_mean'].min()